## Iteractively refine dataset labels

In [54]:
import os
import pathlib
import typing as t
import random
import requests
import json as json_package
import collections

import datasets
import tokenizers
import ipywidgets
import colorama
import tqdm.auto

import segmentador
from config import *
import interactive_labeling


%load_ext autoreload
%autoreload 2


CN = colorama.Fore.RED
CT = colorama.Fore.YELLOW
CLN = colorama.Fore.CYAN
CSR = colorama.Style.RESET_ALL


VOCAB_SIZE = 6000
AUTOSAVE_IN_N_INSTANCES = 20


DATASET_SPLIT = "test"
CACHED_INDEX_FILENAME = "refined_indices.csv"
BRUTE_DATASET_DIR = "../data"
REFINED_DATASET_DIR = os.path.join(BRUTE_DATASET_DIR, "refined_datasets")

TARGET_DATASET_NAME = f"df_tokenized_split_0_120000_{VOCAB_SIZE}"

BRUTE_DATASET_URI = os.path.join(BRUTE_DATASET_DIR, TARGET_DATASET_NAME)
REFINED_DATASET_URI = os.path.join(REFINED_DATASET_DIR, TARGET_DATASET_NAME)
CACHED_INDEX_URI = os.path.join(REFINED_DATASET_DIR, CACHED_INDEX_FILENAME)


assert BRUTE_DATASET_URI != REFINED_DATASET_URI


logit_model = segmentador.BERTSegmenter(
    uri_model=f"../pretrained_segmenter_model/4_{VOCAB_SIZE}_layer_model",
    device="cpu",
)

pbar_dump = tqdm.auto.tqdm(desc="Instances until next dump:", total=AUTOSAVE_IN_N_INSTANCES)

random.seed(17)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Instances until next dump::   0%|          | 0/20 [00:00<?, ?it/s]

In [2]:
tokenizer = tokenizers.Tokenizer.from_file(f"../tokenizers/{VOCAB_SIZE}_subwords/tokenizer.json")
tokenizer.get_vocab_size()

6000

## Load dataset

In [3]:
try:
    df_refined = datasets.load_from_disk(REFINED_DATASET_URI)
    
    with open(CACHED_INDEX_URI, "r") as f_index:
        cached_indices = set(map(int, f_index.read().split(",")))
        
    print("Loaded pre-refined dataset from disk.")

except FileNotFoundError:
    df_refined = datasets.DatasetDict()
    cached_indices = set()
    print("Created new refined dataset.")
    
new_refined_instances = collections.defaultdict(list)
df_brute = datasets.load_from_disk(BRUTE_DATASET_URI)
df_brute, df_refined

Created new refined dataset.


(DatasetDict({
     train: Dataset({
         features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
         num_rows: 114884
     })
     eval: Dataset({
         features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
         num_rows: 14361
     })
     test: Dataset({
         features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
         num_rows: 14361
     })
 }),
 DatasetDict({
     
 }))

## Iteractive refinement

In [108]:
df_split = df_brute[DATASET_SPLIT]

fn_rand = lambda: random.randint(0, df_split.num_rows)
it_counter = 0

df_view_labels = df_split["labels"]
df_view_input_ids = df_split["input_ids"]

cls2id = {
    "no-op": 0,
    "seg": 1,
    "n-start": 2,
    "n-end": 3,
}

def print_labels(input_ids, labels, input_is_tokens: bool = False):
    seg_counter = 1
    
    print(end=CSR)
    print(end=f"{CLN}{seg_counter}.{CSR} ")
    
    if not input_is_tokens:
        tokens = list(map(tokenizer.id_to_token, input_ids))
    
    else:
        tokens = input_ids
    
    for i, (tok, lab) in enumerate(zip(tokens, labels)):
        if lab == cls2id["seg"]:
            seg_counter += 1
            print("\n\n", end=f"{CLN}{seg_counter}.{CSR} ")
        
        if lab == cls2id["n-start"]:
            print(end=CN)
            
        if lab == cls2id["n-end"]:
            print(end=CSR)
            
        print(tok, end=" ")
        
    return tokens


def dump_refined_dataset():
    new_subset = datasets.Dataset.from_dict(new_refined_instances, split=DATASET_SPLIT)
    
    if DATASET_SPLIT not in df_refined:
        df_refined[DATASET_SPLIT] = datasets.Dataset.from_dict({}, split=DATASET_SPLIT)
    
    df_refined[DATASET_SPLIT] = datasets.concatenate_datasets(
        [df_refined[DATASET_SPLIT], new_subset],
        split=DATASET_SPLIT,
    )
    df_refined.save_to_disk(dataset_dict_path=REFINED_DATASET_URI)
    
    new_refined_instances.clear()
    
    print(f"Saved progress in '{REFINED_DATASET_URI}'.")
    
    with open(CACHED_INDEX_URI, "w") as f_index:
        f_index.write(",".join(map(str, sorted(cached_indices))))
    
    it_counter = 0

In [329]:
id_ = fn_rand()
while id_ in cached_indices:
    id_ = fn_rand()


input_ids = df_view_input_ids[id_]
labels = df_view_labels[id_]
tokens = print_labels(input_ids, labels)

1. [CLS] PROJETO DE LEI Nº de 2016 ( Do Sr . Ar ##na ##l ##do F ##ari ##a de S ##á ) 

2. Altera o Código de Defesa do Consumidor , dispon ##do as exigências indispens ##áveis para a realização das ano ##tações neg ##ativa ##s dos consumidores , e a ved ##ação da realização de cobrança de débitos pelos cadastro ##s de proteção ao crédito e congên ##eres . 

3. Art . 1º . Esta lei altera o artigo 43 , da Lei nº 8 . 078 , de 11 de setembro de 1990 . 

4. Art . 2º . Os § § 2º e 4º do artigo 43 , da Lei nº 8 . 078 de 11 de setembro de 1990 , passam a vigorar , altera ##dos , com a seguinte redação : “ 

5. Art . 43 . . . . . . . 

6. § 2° A abertura de cadastro , fic ##ha , registro e dados pessoa ##is e de consumo deverá ser comunic ##ada por escrito ao consumidor , quando não solici ##tada por ele , sendo que as ano ##tações neg ##ativa ##s que não sejam oriun ##das de dívidas prot ##esta ##das ou de cobrança em juízo , só poderão ser realizadas diante : 

7. I – do documento apresentado

In [326]:
logits = logit_model(df_split[id_], return_logits=True).logits
interactive_labeling.open_example(tokens, labels, logits=logits)
lock_save = True

In [327]:
ret = interactive_labeling.retrieve_refined_example()
labels = ret["labels"]
print_labels(tokens, labels, input_is_tokens=True);
lock_save = False

1. [CLS] , distribuição de conteúdo ##s progra ##m ##áticos para a educação de trânsito e promoção e divulgação de trabalhos técnicos sobre trânsito ; 

2. VIII - na promoção da realização de reuniões regionais e cong ##res ##sos nacionais de trânsito , bem como na representação do Brasil em cong ##res ##sos ou reuniões internacionais relacionados com a segurança e educação de trânsito ; 

3. IX - na elaboração e promoção de projetos e programas de formação , tre ##inamento e especial ##ização do pessoal encar ##reg ##ado da execução das atividades de eng ##enha ##ria , educação , informa ##tização , pol ##icia ##mento os ##te ##ns ##iv ##o , fiscalização , operação e administração de trânsito ; 

4. X - na organização e manutenção de modelo pad ##rão de coleta de informações sobre as ocorrência ##s e os acidentes de trânsito ; 

5. XI - na implementação de acordos de cooperação com organ ##ismos internacionais com vista ao aperfeiço ##amento das ações iner ##entes à segurança e educaç

In [330]:
if not lock_save and id_ not in cached_indices:
    it_counter += 1
    cached_indices.add(id_)

    for key, val in df_split[id_].items():
        if key != "labels":
            new_refined_instances[key].append(val.copy())

        else:
            new_refined_instances[key].append(labels)
     
    pbar_dump.update()


if it_counter % AUTOSAVE_IN_N_INSTANCES == 0:
    dump_refined_dataset()
    pbar_dump.reset()

    
print(pbar_dump)

Saved progress in '../data/refined_datasets/df_tokenized_split_0_120000_6000'.
Instances until next dump::   0%|          | 0/20 [00:00<?, ?it/s]
